<h1 style="text-align: center;">
<b>Gastos Hospitalares</b>
</h1>

<h3 style="text-align: justify;">
Script para criação de mapas com dados de gastos hospitalares no Brasil.
</h3>

<p style='text-align: justify;'>
Os dados presentes nas análises foram extraídos do Tabnet. O Tabnet é uma 
ferramenta desenvolvida pelo Departamento de Informação e Informática do Sistema 
Único de Saúde (DataSUS) da Secretaria de Informação e Saúde Digital do Ministério
da Saúde (MS). Esta ferramenta abrange dados epidemiológicos, de mortalidade, 
da rede assistencial, de internações hospitalares, de procedimentos realizados 
na atenção primária, entre outras.
Para este trabalho, foram usadas as informações do Sistema de Informações 
Hospitalares (SIH), referentes às internações hospitalares aprovadas no período 
de 2014 a 2022. Foram usados os dados de Autorização de Internação Hospitalar 
(AIH) no formato reduzido (RD) com abrangência geográfica no nível municipal. 
Foi selecionado para o campo linha os municípios brasileiros. Para as colunas, 
foi selecionado o ano/mês de processamento.
</p>

<h4 style="text-align: justify;">
<b>Gastos Hospitalares</b>
<h4>

<p style="text-align: justify;">
O valor do gasto hospitalar representa o valor total dos procedimentos presentes na AIH realizados naquela internação. Esse valores são regulamentados pela Tabela Unificada de Procedimentos, Medicamentos e Órteses, Próteses e Materiais Especiais do SUS (SIGTAP).
</p>

<h4 style="text-align: justify;">
<b>Mortalidade</b>
<h4>

<p style="text-align: justify;">
A taxa de mortalidade é calculada pela razão entre o número de óbitos e número de internações, dada também pelo TabNet.
</p>

<h4 style="text-align: justify;">
<b>População</b>
<h4>
<p style='text-align: justify;'>
Os dados de população dos municípios utilizados neste trabalho, foram obtidas a partirde duas fontes. Para os anos de 2014 a 2021, foram utilizadas as estimativas preliminares
elaboradas pelo Ministério da Saúde, disponibilizados pelo TabNet.
</p>
<p style='text-align: justify;'>
O dado de população em 2022 foi obtido no site do Instituto Brasileiro de Geografia eEstatística - IBGE.
</p>

<p style="text-align: justify;">
<b>Acesse os dados no portal</b>

[Datasus Saúde Governo Federal](https://datasus.saude.gov.brinformacoes-de-saude-tabnet/)
</p>

<p style="text-align: justify;">

[Panorama Censo 2022 - IBGE](https://censo2022.ibge.gov.br/panorama/)
</p>

<p style="text-align: justify;">
___________________________________________


In [ ]:
import pandas as pd
import requests
import plotly.express as px
from geojson_rewind import rewind


In [ ]:
df = pd.read_excel('base_tratada.xlsx', engine='openpyxl')

df.head()

In [ ]:
# gráficos dinâmicos com plotly


# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig = px.choropleth(df,
                    geojson=geojson,
                    scope='south america',
                    color='gastos',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=50_000,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 1_560_000_000],  # Defina manualmente o intervalo de cores
                    locations='cod_ibge',
                    featureidkey='properties.codarea',
                    hover_name='municipio',
                   animation_frame='ano')

fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(marker_line_width=0)

fig.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 300, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig.show()